<a href="https://colab.research.google.com/github/YuXuan20040221/CNN-Assignment-2025/blob/main/RoadDamage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 道路安全

注意事項:

* 需要用到GPU到 執行階段 > 變更執行階段類型去設定
* 打 "*" 號的地方是剛開始(打開colab)時需要執行的
* 要換資料集到"資料集設定" (旁邊的目錄有寫)
* 設定修改之後要先跑一次讓他寫進檔案才有效
* 調整參數到"訓練"


# *掛載 Google Drive
讓 Colab 讀取放在雲端硬碟的資料集
(這邊不用動)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# *安裝相關套件

In [3]:
!pip install ultralytics

  Using cached ultralytics-8.3.163-py3-none-any.whl.metadata (37 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.

## 掛載Google sheet

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive"
]

creds = ServiceAccountCredentials.from_json_keyfile_name(
    "/content/drive/MyDrive/credentials.json", scope
)

client = gspread.authorize(creds)
sheet = client.open("model_test_result").worksheet("YOLO_params")


# 設定(yaml)

## 資料集設定:

自行修改資料集目錄路徑

In [4]:
# 定義train.yaml ->
yaml_content = """
path: /content/drive/MyDrive/datasets  # 整體資料集根目錄
train: Processed/TLEF/images # 訓練用資料夾位置
val: Processed/T/images # 驗證用資料夾位置

nc: 1 # 類別個數
names: ["pothole"] # 類別名稱
"""

# 寫入雲端 ->
with open("/content/drive/MyDrive/datasets/train.yaml", "w") as f:
    f.write(yaml_content)

## 模型架構設定:

(這我自己在玩的不用鳥他)

```python
# YOLOv8n 官方模型架構

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  8                  -1  1    460288  ultralytics.nn.modules.block.C2f             [256, 256, 1, True]           
  9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  1    148224  ultralytics.nn.modules.block.C2f             [384, 128, 1]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  1     37248  ultralytics.nn.modules.block.C2f             [192, 64, 1]                  
 16                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
 17            [-1, 12]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 18                  -1  1    123648  ultralytics.nn.modules.block.C2f             [192, 128, 1]                 
 19                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
 20             [-1, 9]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 21                  -1  1    493056  ultralytics.nn.modules.block.C2f             [384, 256, 1]                 
 22        [15, 18, 21]  1    751507  ultralytics.nn.modules.head.Detect           [1, [64, 128, 256]]           

Model summary: 129 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
```

In [ ]:
# 參考自: https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/models/v8/yolov8.yaml
# 僅調整head部分(backbone不動)

# yaml檔 ->
yaml_content = """
# Ultralytics 🚀 AGPL-3.0 License - https://ultralytics.com/license

# Ultralytics YOLOv8 object detection model with P3/8 - P5/32 outputs
# Model docs: https://docs.ultralytics.com/models/yolov8
# Task docs: https://docs.ultralytics.com/tasks/detect

# Parameters
nc: 1 # number of classes(類別數量)
scales: # model compound scaling constants, i.e. 'model=yolov8n.yaml' will call yolov8.yaml with scale 'n'
  # [depth, width, max_channels]
  n: [0.33, 0.25, 1024] # YOLOv8n summary: 129 layers, 3157200 parameters, 3157184 gradients, 8.9 GFLOPS
  s: [0.33, 0.50, 1024] # YOLOv8s summary: 129 layers, 11166560 parameters, 11166544 gradients, 28.8 GFLOPS
  m: [0.67, 0.75, 768] # YOLOv8m summary: 169 layers, 25902640 parameters, 25902624 gradients, 79.3 GFLOPS
  l: [1.00, 1.00, 512] # YOLOv8l summary: 209 layers, 43691520 parameters, 43691504 gradients, 165.7 GFLOPS
  x: [1.00, 1.25, 512] # YOLOv8x summary: 209 layers, 68229648 parameters, 68229632 gradients, 258.5 GFLOPS

# YOLOv8.0n backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]] # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]] # 1-P2/4
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]] # 3-P3/8
  - [-1, 6, C2f, [256, True]]
  - [-1, 1, Conv, [512, 3, 2]] # 5-P4/16
  - [-1, 6, C2f, [512, True]]
  - [-1, 1, Conv, [1024, 3, 2]] # 7-P5/32
  - [-1, 3, C2f, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]] # 9

# YOLOv8.0n head
head:
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 6], 1, Concat, [1]] # cat backbone P4
  - [-1, 4, C2f, [256]] # 12

  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 4], 1, Concat, [1]] # cat backbone P3
  - [-1, 3, C2f, [256]] # 15 (P3/8-small)

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 12], 1, Concat, [1]] # cat head P4
  - [-1, 3, C2f, [512]] # 18 (P4/16-medium)

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 9], 1, Concat, [1]] # cat head P5
  - [-1, 5, C2f, [512]] # 21 (P5/32-large)

  - [[15, 18, 21], 1, Detect, [nc]] # Detect(P3, P4, P5)
"""

# 寫入雲端硬碟檔案 ->
with open("/content/drive/MyDrive/datasets/custom_yolov8n.yaml", "w") as f:
    f.write(yaml_content)

# 訓練
參數調整參考:
[YOLO調整指南](https://docs.ultralytics.com/zh/guides/hyperparameter-tuning/)

In [5]:
import os
import torch
from ultralytics import YOLO

# 建立模型

# 模型名稱: 可用YOLO原始模型、自定義模型、之前練過的
model = YOLO("yolov8n.pt")

# 我自己要玩的，不想用就幫我註解掉就好(刪了我會哭嗚嗚)

# # 1. 官方模型(抓預訓練權重用)
# pretrained = YOLO("yolov8n.pt")
# # 2. 自訂模型架構
# model = YOLO("/content/drive/MyDrive/datasets/custom_yolov8n.yaml")
# # 3. 取得預訓練模型的state_dict
# pretrained_dict = pretrained.model.state_dict()
# new_model_dict = model.model.state_dict()
# # 4. 把對得上的 key 丟進去（只留 backbone 的部分）
# filtered_dict = {k: v for k, v in pretrained_dict.items() if k in new_model_dict and v.shape == new_model_dict[k].shape}
# # 5. 套用進去
# model.model.load_state_dict(filtered_dict, strict=False)

# 訓練模型
model.train(
    # 訓練用參數
    data="/content/drive/MyDrive/datasets/train.yaml",
    epochs=150, # 圈數
    batch=32, # 單次處理影像數量
    imgsz=640, # 圖片大小
    project="/content/drive/MyDrive/model_custom1", # 存在哪資料夾
    name="yolov8n_TLEF_1", # 叫啥名
    # lr0=0.001, # 初始學習率(越小收斂速度越慢但越穩)
    # lrf=0.01, # 最終學習率(是lr0的幾%，控制下降幅度)
    # weight_decay=0.0005, # L2，防overfitting(但調太大模型難學)
    # optimizer="AdamW", # 優化器(AdamW好像大家都會用)
    # patience=50,
    # warmup_epochs=5, # 穩定前幾epoch
    # dropout=0.2, # 關掉多少神經元
    device='cuda',

    # 圖片處理
    # hsv_h=0.015,     # 色相擾動
    # hsv_s=0.7,       # 飽和度擾動
    # hsv_v=0.4,       # 亮度擾動
    # degrees=10.0,    # 旋轉
    # translate=0.1,   # 平移
    # scale=0.3,       # 縮放
    # shear=5.0,       # 剪切
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 360MB/s]


Ultralytics 8.3.163 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/datasets/train.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_TLEF_1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, 

100%|██████████| 755k/755k [00:00<00:00, 116MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all         79        135      0.944      0.904      0.949      0.659



150 epochs completed in 0.137 hours.
Optimizer stripped from /content/drive/MyDrive/model_custom1/yolov8n_TLEF_1/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/model_custom1/yolov8n_TLEF_1/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/model_custom1/yolov8n_TLEF_1/weights/best.pt...
Ultralytics 8.3.163 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.31it/s]


                   all         79        135      0.955      0.946      0.976      0.694
Speed: 0.1ms preprocess, 2.3ms inference, 0.0ms loss, 4.0ms postprocess per image
Results saved to /content/drive/MyDrive/model_custom1/yolov8n_TLEF_1


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b3d7acdd550>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

# 測試
(不用管他就是我試試看用的)

In [ ]:
from ultralytics import YOLO

model = YOLO("/content/drive/MyDrive/model_n/yolov8n_Processed_2_1_1/weights/best.pt")

results = model.predict(
    source="/content/drive/MyDrive/datasets/Processed_1/valid/images",# ← 測試圖片資料夾
    conf=0.32,          # ← 降低信心閾值，提高 Recall
    iou=0.45,          # ← 放寬 NMS iou
    save=True,          # ← 會把結果圖片存起來
    save_txt=True,        # ← 存文字框框檔
    augment=True         # ← 開啟推論增強（可略）
)

model.val(data="/content/drive/MyDrive/datasets/train.yaml", conf=0.1) # 評估報告


image 1/57 /content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000048.jpg: 640x640 1 pothole, 800.2ms
image 2/57 /content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000057.jpg: 640x640 5 potholes, 566.8ms
image 3/57 /content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000061.jpg: 640x640 1 pothole, 583.2ms
image 4/57 /content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000062.jpg: 640x640 4 potholes, 552.3ms
image 5/57 /content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000064.jpg: 640x640 1 pothole, 499.6ms
image 6/57 /content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000136.jpg: 640x640 (no detections), 550.6ms
image 7/57 /content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000160.jpg: 640x640 1 pothole, 480.6ms
image 8/57 /content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000205.jpg: 640x640 3 potholes, 533.9ms
image 9/57 /content/drive/MyDrive/data

100%|██████████| 755k/755k [00:00<00:00, 19.0MB/s]


val: Fast image access ✅ (ping: 0.6±0.2 ms, read: 0.3±0.1 MB/s, size: 62.8 KB)


val: Scanning /content/drive/MyDrive/datasets/China/valid/labels.cache... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:17<00:00,  8.98s/it]


                   all         25         34      0.596      0.348      0.422        0.2
Speed: 17.5ms preprocess, 295.5ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/val


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7faf6e7b7390>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

## 測overfitting

In [ ]:
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/model_n/yolov8n_Origin_3_2/weights/best.pt")  # 換成你的模型路徑


# 訓練集評估
train_metrics = model.val(data="/content/drive/MyDrive/datasets/train.yaml", split="train", conf=0.001, iou=0.6)

# 驗證集評估
val_metrics = model.val(data="/content/drive/MyDrive/datasets/train.yaml", split="val", conf=0.001, iou=0.6)

def compute_f1(precision, recall):
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)

# 訓練集 metrics
train_precision = train_metrics.results_dict['metrics/precision(B)']
train_recall = train_metrics.results_dict['metrics/recall(B)']
train_f1 = compute_f1(train_precision, train_recall)

# 驗證集 metrics
val_precision = val_metrics.results_dict['metrics/precision(B)']
val_recall = val_metrics.results_dict['metrics/recall(B)']
val_f1 = compute_f1(val_precision, val_recall)

# 差異比較
print(f"Train F1: {train_f1:.3f}")
print(f"Val F1:   {val_f1:.3f}")
print(f"F1 GAP :  {train_f1 - val_f1:.3f}")


Ultralytics 8.3.141 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.4±0.2 ms, read: 48.0±12.9 MB/s, size: 137.2 KB)


val: Scanning /content/drive/MyDrive/datasets/Origin_3/train/labels.cache... 165 images, 8 backgrounds, 0 corrupt: 100%|██████████| 173/173 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:03<00:00,  2.93it/s]


                   all        173        235       0.97      0.979      0.972       0.56
Speed: 1.2ms preprocess, 6.5ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs/detect/val11
Ultralytics 8.3.141 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.3±0.1 ms, read: 27.0±14.5 MB/s, size: 58.1 KB)


val: Scanning /content/drive/MyDrive/datasets/Origin_3/valid/labels.cache... 27 images, 8 backgrounds, 0 corrupt: 100%|██████████| 35/35 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.25it/s]


                   all         35         34       0.52      0.588      0.466      0.253
Speed: 10.9ms preprocess, 6.5ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs/detect/val12
Train F1: 0.975
Val F1:   0.552
F1 GAP :  0.423


# 投票

嬿雯讓我試但我還沒成功

## 聯集(OR)

In [ ]:
from ultralytics import YOLO
from pathlib import Path
import torch
import numpy as np

# 載入模型
model_a = YOLO("/content/drive/MyDrive/model_n/yolov8n_Processed_2_1_1/weights/best.pt")
model_b = YOLO("/content/drive/MyDrive/model_n/yolov8n_OriData_4_1/weights/best.pt")

# 預測資料夾
img_dir = Path("/content/drive/MyDrive/datasets/Processed_1/valid/images")
output_dir = Path("/content/drive/MyDrive/datasets/voted_preds/labels")
output_dir.mkdir(parents=True, exist_ok=True)

# -------- Utils --------
def yolo_to_xyxy_tensor(boxes, img_w, img_h):
    """YOLO format (cx, cy, w, h) to XYXY with clamping, tensor version"""
    cx = boxes[:, 0] * img_w
    cy = boxes[:, 1] * img_h
    w = boxes[:, 2] * img_w
    h = boxes[:, 3] * img_h

    x1 = (cx - w / 2).clamp(0, img_w)
    y1 = (cy - h / 2).clamp(0, img_h)
    x2 = (cx + w / 2).clamp(0, img_w)
    y2 = (cy + h / 2).clamp(0, img_h)

    return torch.stack([x1, y1, x2, y2], dim=1)

# 輸出融合後的結果（A or B 模式）
for img_path in img_dir.glob("*.jpg"):
    result_a = model_a.predict(str(img_path), conf=0.32, iou=0.4, verbose=False)[0]
    result_b = model_b.predict(str(img_path), conf=0.353, iou=0.4, verbose=False)[0]

    all_boxes = []

    # 把每個模型的預測框轉換為YOLO格式
    for r in [result_a, result_b]:
        if r.boxes is not None:
            for box in r.boxes:
                cls = int(box.cls[0].item())
                conf = float(box.conf[0].item())
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                x_center, y_center, w, h = box.xywhn[0].tolist()
                all_boxes.append([cls, x_center, y_center, w, h, conf])

    # 把 YOLO 格式轉換為 tensor (XYXY 格式 + conf + cls)
    if all_boxes:
        boxes_tensor = torch.tensor(all_boxes)  # [cls, x_center, y_center, w, h, conf]
        conf = boxes_tensor[:, 5]
        cls = boxes_tensor[:, 0]
        xywh = boxes_tensor[:, 1:5]

        # 轉換為 xyxy
        img_w, img_h = Image.open(img_path).size
        xyxy = yolo_to_xyxy_tensor(xywh, img_w, img_h)

        # 做 NMS（使用 torchvision 自帶的函數）
        import torchvision
        keep = torchvision.ops.nms(xyxy, conf, iou_threshold=0.5)

        # 過濾出保留的框
        filtered_boxes = boxes_tensor[keep]
    else:
        filtered_boxes = torch.zeros((0, 6))


    # 寫入 fusion 預測檔
    label_path = output_dir / f"{img_path.stem}.txt"
    with open(label_path, "w") as f:
        # for box in all_boxes:
        for box in filtered_boxes:
            f.write(f"{box[0]} {box[1]:.6f} {box[2]:.6f} {box[3]:.6f} {box[4]:.6f} {box[5]:.6f}\n")

    result = model_a.predict(img_path, conf=0.3, iou=0.4, save=False, verbose=False)[0]
    for box in result.boxes:
        print(f"xywhn{img_path}:", box.xywhn[0].tolist())


xywhn/content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000286.jpg: [0.22385621070861816, 0.7224076390266418, 0.1443481296300888, 0.37175655364990234]
xywhn/content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000057.jpg: [0.4367590546607971, 0.48660576343536377, 0.09849473834037781, 0.10497763752937317]
xywhn/content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000057.jpg: [0.9770104885101318, 0.26265737414360046, 0.045785725116729736, 0.11710286140441895]
xywhn/content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000057.jpg: [0.9698253870010376, 0.050241950899362564, 0.06022912263870239, 0.0781131461262703]
xywhn/content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000057.jpg: [0.4313870072364807, 0.7975969314575195, 0.07193639874458313, 0.0777050256729126]
xywhn/content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000160.jpg: [0.9319285154342651, 0.2945277690887451, 0.06749075651168823, 0.06

## 驗證

就是跑看看投票結果長怎樣

✅會比較慢所以之後雲端可以用

In [ ]:
from pathlib import Path
import torch
from ultralytics.utils.metrics import ConfusionMatrix
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# -------- Utils --------
def yolo_to_xyxy_tensor(boxes, img_w, img_h):
    """YOLO format (cx, cy, w, h) to XYXY with clamping, tensor version"""
    cx = boxes[:, 0] * img_w
    cy = boxes[:, 1] * img_h
    w = boxes[:, 2] * img_w
    h = boxes[:, 3] * img_h

    x1 = (cx - w / 2).clamp(0, img_w)
    y1 = (cy - h / 2).clamp(0, img_h)
    x2 = (cx + w / 2).clamp(0, img_w)
    y2 = (cy + h / 2).clamp(0, img_h)

    return torch.stack([x1, y1, x2, y2], dim=1)

# -------- Visualization --------
def visualize_boxes(img_path, gt_boxes, pred_boxes, save_path):
    img = Image.open(img_path).convert("RGB")
    fig, ax = plt.subplots(1, figsize=(12, 12))
    ax.imshow(img)

    # 畫 Ground Truth 框（綠色）
    for box in gt_boxes:
        x1, y1, x2, y2 = box.tolist()
        w, h = x2 - x1, y2 - y1
        rect = patches.Rectangle((x1, y1), w, h, linewidth=2, edgecolor='lime', facecolor='none')
        ax.add_patch(rect)

    # 畫 Prediction 框（紅色）
    for box in pred_boxes:
        x1, y1, x2, y2 = box.tolist()
        w, h = x2 - x1, y2 - y1
        rect = patches.Rectangle((x1, y1), w, h, linewidth=2, edgecolor='red', facecolor='none')
        ax.add_patch(rect)

    ax.axis("off")
    fig.tight_layout()
    fig.savefig(save_path)
    plt.close(fig)


# -------- Init --------
gt_path = Path("/content/drive/MyDrive/datasets/Processed_1/valid/labels")
pred_path = Path("/content/drive/MyDrive/datasets/voted_preds/labels")
img_dir = Path("/content/drive/MyDrive/datasets/Processed_1/valid/images")  # ← 確保這是你圖片目錄

vis_dir = Path("/content/drive/MyDrive/datasets/voted_preds/vis")  # 儲存可視化結果
vis_dir.mkdir(parents=True, exist_ok=True)

cm = ConfusionMatrix(nc=1)  # 只有一個類別

# -------- Evaluation Loop --------
for gt_file in gt_path.glob("*.txt"):
    pred_file = pred_path / gt_file.name
    img_path = img_dir / (gt_file.stem + ".jpg")
    img_w, img_h = Image.open(img_path).size

    # --- Load GT ---
    gt_lines = gt_file.read_text().strip().splitlines()
    gt = torch.tensor([list(map(float, l.split())) for l in gt_lines]) if gt_lines else torch.zeros((0, 5))
    gt_cls = gt[:, 0] if len(gt) else torch.zeros((0,))
    gt_boxes = yolo_to_xyxy_tensor(gt[:, 1:5], img_w, img_h) if len(gt) else torch.zeros((0, 4))

    # --- Load Prediction ---
    if pred_file.exists():
        pred_lines = pred_file.read_text().strip().splitlines()
        pred = torch.tensor([list(map(float, l.split())) for l in pred_lines]) if pred_lines else torch.zeros((0, 6))
        pred_cls = pred[:, 0]
        pred_boxes = yolo_to_xyxy_tensor(pred[:, 1:5], img_w, img_h)
        pred_conf = pred[:, 5]
    else:
        pred_cls = torch.zeros((0,))
        pred_boxes = torch.zeros((0, 4))
        pred_conf = torch.zeros((0,))

    # --- Format Detections ---
    if len(pred_boxes):
        detections = torch.cat([pred_boxes, pred_conf[:, None], pred_cls[:, None]], dim=1)
    else:
        detections = torch.zeros((0, 6))

    # --- Process ---
    if len(gt_boxes) or len(detections):
        cm.process_batch(detections, gt_boxes, gt_cls)
        # --- 可視化目前這張圖片的 GT 和預測框 ---
        save_img_path = vis_dir / f"{gt_file.stem}_vis.jpg"
        visualize_boxes(img_path, gt_boxes, pred_boxes, save_img_path)

    # --- Optional Debug ---
    print(f"\n📄 {gt_file.name}")
    print(f"GT cls:     {gt_cls.shape} {gt_cls}")
    print(f"GT boxes:   {gt_boxes.shape} {gt_boxes}")
    print(f"Detections: {detections.shape}")

# -------- Metrics --------
confmat = cm.matrix
eps = 1e-16
TP = confmat.diagonal()
FP = confmat.sum(0) - TP
FN = confmat.sum(1) - TP

precision = (TP / (TP + FP + eps)).mean()
recall = (TP / (TP + FN + eps)).mean()
f1 = 2 * precision * recall / (precision + recall + eps)

print(f"\n📊 Voting Ensemble Evaluation Results:")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"F1 Score:  {f1:.3f}")



📄 China_Drone_000048.txt
GT cls:     torch.Size([1]) tensor([0.])
GT boxes:   torch.Size([1, 4]) tensor([[216.,  85., 285., 280.]])
Detections: torch.Size([1, 6])

📄 China_Drone_000435.txt
GT cls:     torch.Size([1]) tensor([0.])
GT boxes:   torch.Size([1, 4]) tensor([[480., 390., 512., 423.]])
Detections: torch.Size([2, 6])

📄 China_Drone_000160.txt
GT cls:     torch.Size([1]) tensor([0.])
GT boxes:   torch.Size([1, 4]) tensor([[ 98.,  32., 136.,  67.]])
Detections: torch.Size([1, 6])

📄 China_Drone_000205.txt
GT cls:     torch.Size([1]) tensor([0.])
GT boxes:   torch.Size([1, 4]) tensor([[134.,  69., 179., 125.]])
Detections: torch.Size([2, 6])

📄 China_Drone_000136.txt
GT cls:     torch.Size([1]) tensor([0.])
GT boxes:   torch.Size([1, 4]) tensor([[271.,  37., 368., 105.]])
Detections: torch.Size([1, 6])

📄 China_Drone_000451.txt
GT cls:     torch.Size([1]) tensor([0.])
GT boxes:   torch.Size([1, 4]) tensor([[ 27., 149.,  64., 180.]])
Detections: torch.Size([1, 6])

📄 China_Drone_0